In [1]:
import sys
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard

from matplotlib import pyplot as plt

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-10 00:48:16,139 log.py               31   [INFO] [__init__] skip wandb init 


## e044 の 1st fold で error analysis してみる

In [3]:
prep_df = dr.load_preprocessed_df(
          dataset_name="train",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV6",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

2021-11-10 00:48:17,969 data_repository.py   253  [INFO] [load_preprocessed_df] loading data/preprocessed/train_BaselineKernelPreprocessorV6_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-10 00:48:20,803 data_repository.py   260  [INFO] [load_preprocessed_df] done. 


In [4]:
best_checkpoints = dr.best_checkpoint_filepaths("e042")
print(best_checkpoints)
# checkpoint = dr.load_checkpoint_from_filepath(filepath_from_root=best_checkpoints[0], load_from_gcs=True, rm_local_after_load=True) 
checkpoint = dr.load_checkpoint_from_filepath(filepath_from_root=best_checkpoints[0], load_from_gcs=True, rm_local_after_load=True) 
dir(checkpoint)

2021-11-10 00:48:20,940 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e042/best_checkpoint/0_3_1.1404_0.6902.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e042/best_checkpoint/0_3_1.1404_0.6902.pkl 


['data/checkpoint/e042/best_checkpoint/0_3_1.1404_0.6902.pkl']


2021-11-10 00:49:13,292 repository.py        176  [INFO] [__download_from_gcs] download done. 


['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'epoch',
 'exp_id',
 'extend_str_list_val_info',
 'extend_tensor_val_info',
 'fold',
 'model_state_dict',
 'non_filled_mambers',
 'optimizer_state_dict',
 'scheduler_state_dict',
 'set_model',
 'set_optimizer',
 'set_scheduler',
 'val_end_logits',
 'val_ids',
 'val_jaccard',
 'val_loss',
 'val_pospro_answer_preds',
 'val_pospro_answer_texts',
 'val_pospro_ids',
 'val_segmentation_logits',
 'val_start_logits']

In [5]:
ids = checkpoint.val_pospro_ids
answer_preds = checkpoint.val_pospro_answer_preds
answer_texts = checkpoint.val_pospro_answer_texts
jaccards = [jaccard(answer_pred, answer_text) for answer_pred, answer_text in zip(answer_preds, answer_texts)]

In [7]:
from time import sleep

fold_df = pd.DataFrame()
fold_df["id"] = ids
fold_df["answer_pred"] = answer_preds
fold_df["answer_text"] = answer_texts
fold_df["jaccard"] = jaccards

fold_df = fold_df.merge(prep_df[["id", "language"]].drop_duplicates(), on="id", how="left")
# fold_df = fold_df.merge(e016_preprocessed_df[["id", "language", "question"]].drop_duplicates(), on="id", how="left")
# fold_df["en_question"] = fold_df.apply(lambda row: translator.translate(row["question"], src=row["language"], dest="en"), axis=1)
# fold_df["en_answer_text"] = fold_df.apply(lambda row:  translator.translate(row["answer_text"], src=row["language"], dest="en"), axis=1)
# fold_df["en_answer_pred"] = fold_df.apply(lambda row: translator.translate(row["answer_pred"], src=row["language"], dest="en"), axis=1)
# fold_df = fold_df[["id", "jaccard", "en_question", "en_answer_text", "en_answer_pred", "answer_text", "answer_pred"]]

In [8]:
# . で終わる場合って消してよいの？どうなの？ ex. id 2af249652 →　数字 + . の場合は消して良さそう
# 大文字小文字問題 ex. a2690fe4c の इस्तानबुल vs इस्तांबुल = istanbul vs Istanbul
# 
fold_df.sort_values("jaccard")

,id,answer_pred,answer_text,jaccard,language
0,01a91f15a,"1,250",सभी पक्षियो के 12.6%,0.000000,hindi
42,3fe627e15,"7,258 மீட்டர்","10,911",0.000000,tamil
43,4035479ab,भारत,हिन्द आर्य,0.000000,hindi
52,51579a4f9,शूरसेन,महाराजा वल्लभ सेन,0.000000,hindi
191,e0090c270,சீனா,பெரும்,0.000000,tamil
186,dab6fa1cf,எரிபொருளாகவும் இதர வேதிப்பொருட்களைத் தயாரிப்ப...,C2H5-OH,0.000000,tamil
183,d7592882c,\nசப்பான்,ஜப்பான்,0.000000,tamil
68,5f7c6e0a7,-1799),1799,0.000000,hindi
142,ad90053ce,"ஜனவரி 10, 2008",2009 ஆம் ஆண்டு மார்ச் 23,0.000000,tamil
178,d3b0f2bc8,मेगाफ्लॉप,मेगा फ्लॉप,0.000000,hindi


In [20]:
# this might be the key point
prep_df.query('max_successed_duplicated_elems_num_with == duplicated_elems_num_with').groupby("id").is_contain_answer_text.mean().value_counts().sort_index()

0.026316      1
0.028571      1
0.029412      2
0.033333      1
0.035714      1
0.040000      1
0.045455      1
0.048780      1
0.050000      3
0.058824      2
0.060606      1
0.066667      3
0.071429      2
0.076923      4
0.083333      6
0.090909      7
0.095238      2
0.100000     10
0.111111      8
0.117647      2
0.125000     13
0.133333      3
0.142857     16
0.153846      2
0.166667     25
0.181818      1
0.200000     31
0.216216      1
0.222222      8
0.230769      1
0.250000     42
0.266667      1
0.272727      2
0.285714     15
0.300000      1
0.333333     85
0.400000     13
0.428571      2
0.500000    137
0.555556      1
0.600000      6
0.666667     29
0.714286      2
0.750000     13
0.857143      1
0.923077      1
1.000000    603
Name: is_contain_answer_text, dtype: int64

In [22]:
prep_df.query('max_successed_duplicated_elems_num_with == duplicated_elems_num_with').groupby("id").is_contain_answer_text.mean().sort_values()

id
6108306f5    0.026316
021fcb4f4    0.028571
dbf1301d6    0.029412
3f3c2c4c5    0.029412
ebeb02880    0.033333
               ...   
7022fc3df    1.000000
702cf6f46    1.000000
7040d2051    1.000000
70e9abc7e    1.000000
ffda743a2    1.000000
Name: is_contain_answer_text, Length: 1114, dtype: float64

In [25]:
# invalid ids = ["01a91f15a", ]
# simply miss = ["3fe627e15", ""]
prep_df.query('id == "6108306f5"')

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with
6879,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,3,203,213,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3
6880,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,3,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6881,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,4,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6882,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6883,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,3,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6884,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",5,3,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6885,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",6,3,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6886,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। विकिपी...,विकिपीडिया के संस्थापक कौन है?,जिम्मी वेल्स और लेरी सेंगर,769,hindi,"[0, 132400, 287, 178208, 49329, 460, 32, 2, 2,...","[None, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",7,3,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
6887,6108306f5,यह लेख इंटरनेट विश्वकोश के बारे में है। वि

In [14]:
row = prep_df.loc[491]

In [15]:
print(row["context"])

கடல் (ஒலிப்பு)  அல்லது ஆழி(Sea), உலகப் பெருங்கடல் (World ocean), அல்லது வெறுமனே பெருங்கடல் (Ocean) என்பது புவியின் பரப்பில் 70 விழுக்காட்டை ஆக்கிரமித்துள்ள உப்பான நீர் கொண்ட இணைந்த (connected) நீர்நிலை ஆகும். இது புவியின் பருவநிலையை நிலைப்படுத்துவதோடு நீர் சுழற்சி, கரிமச் சுழற்சி, நைட்ரஜன் சுழற்சி ஆகியவற்றிலும் முதன்மைப் பங்காற்றுகிறது. பழங்காலங்களிலிருந்து கடலில் பயணங்கள் செய்யப்பட்டும் தேடல்கள் நடந்தும் வந்தாலும், அறிவியல் அடிப்படையிலான கடலியல் அல்லது பெருங்கடலியல் என்பது பெரும்பாலும் பசிபிக் பெருங்கடலை ஜேம்ஸ் குக் 1768க்கும் 1779க்கும் இடைப்பட்ட காலத்தில் கண்டறிந்து ஆராய்ந்ததிலிருந்துதான் தொடங்குகிறது. கடல் எனும் சொல்லானது பெருங்கடலின் சிறிய, பகுதியளவு நிலத்தால் சூழப்பட்ட பகுதிகளைக் குறிக்கவும் பயன்படுத்தப்படுகிறது."கடல்" என்ற சொல்லாவது, கடத்தற்கு அரியதென்று பொருள்படும். ஆழி, விரிநீர், பெருநீர், பருநீர், முதலானும் குறிக்கப்படுகிறது.
கடல் நீரில் மிக அதிகளவு கரைந்துள்ள திடப்பொருள் சோடியம் குளோரைடு (சாதாரண உப்பு) ஆகும். மேலும் இந்நீரில் மக்னீசியம், கால்சியம், பொட்டாசியம் போன்ற உப்புகளு